In [1]:
import pickle
from sentence_transformers import SentenceTransformer


In [ ]:
# VECTOR_SPACE_PATH = "/kaggle/input/chaabi/space.pkl"
BERT_MODEL_PATH = "bert-question-answering.pkl"
ENC_PATH  = "encodermodel.pkl"
QDRANT_PATH  = "qdrant_vectorsdb_client.pkl"


# with open(VECTOR_SPACE_PATH, 'rb') as file:
#     vs = pickle.load(file)
# bert llm
with open(BERT_MODEL_PATH, 'rb') as file:
    bert = pickle.load(file)


    
    

In [ ]:
with open(QDRANT_PATH, 'rb') as file:
    qdrant_client = pickle.load(file)

with open(ENC_PATH, 'rb') as file:
    st_encoder = pickle.load(file)
    

In [ ]:
collection_name = "qdrant-space"
def find_close_contexts(question: str, top_k: int) -> list[str]:
    """
    will return contexts contexts close to query 

    Args:
        question (str): What do we want to know?
        top_k (int): top k results will be added 

    Returns:
        context (List[str]):
    """
    try:
        encoded_query = st_encoder.encode(question).tolist() 
        result = qdrant_client.search(
            collection_name=collection_name,
            query_vector=encoded_query,
            limit=top_k,
        )  # search qdrant collection for context passage with the answer

        context = [
            [context.payload["product"], context.payload["story"]] for context in result
        ] 
        return context
    except Exception as e:
        print({e})

In [ ]:
def tell_me(question: str, context: list[str]):
    """
    Extract the answer from the context for a given question

    Args:
        question (str): _description_
        context (list[str]): _description_
    """
    results = []
    for c in context:
        answer = bert(question=question, context=c[1] )
        answer["product"] = c[0]
        results.append(answer)
        print()

    sorted_result = sorted(results, key=lambda x: x["score"], reverse=True)
    for i in range(len(sorted_result)):
        _out = sorted_result[i]["answer"] 
        _prod = sorted_result[i]["product"]
        _sco = sorted_result[i]["score"]
#         print(f"{i+1}", end=" ")
        print(f"QUERY INPUT: {question}")
        print(f"OUTPUT: {_out} \nPREDICTION SCORE {_sco}\n\nReferred Product: {_prod}\n\n")
        return question,_out,_sco,_prod

In [ ]:
queries = []
queries.append("suggest me one product for cleaning vegetables")
queries.append("what is the rating of product Vegetable & Fruit Wash with 100% Natural Action")
queries.append("what is most loved beauty product")
queries.append("price of dove soap")
queries.append("what is most loved beauty product")
queries.append("suggest one Tea Product")

In [ ]:
with open("result.txt","w") as result:
    for q in queries:
        
        c = find_close_contexts(q, top_k=1)
        _ques,_out,_sco,_prod = tell_me(q,c)
        result.write(f"QUERY INPUT: {_ques}\nOUTPUT: {_out} \nPREDICTION SCORE {_sco}\n\nReferred Product: {_prod}\n\n")
    